In [3]:
'''
These are the important imports and functions used in the detection part of the project
assuming the other libraries are already installed and imported like cv2, numpy, etc.
'''
from fastai.vision.all import *
from fastai.metrics import accuracy
from fastseg import MobileV3Small
import torch

import cv2
import numpy as np
from scipy.spatial import distance

In [4]:
# The functions needed for the detection part
def get_pred_for_mobilenet(model, img_array):
    # this is to make sure that the model is in the GPU
    with torch.no_grad():
        # image_tensor = np.expand_dims(img_array, -1).transpose(2, 0, 1).astype('float32') / 255
        image_tensor = img_array.transpose(2, 0, 1).astype("float32") / 255
        x_tensor = torch.from_numpy(image_tensor).to("cuda").unsqueeze(0)
        model_output = F.softmax(model.forward(x_tensor), dim=1).cpu().numpy()
    return model_output


def extract_lane_boundaries(left_mask, right_mask, threshold=0.3):
    left_boundary = (left_mask > threshold).astype(np.uint8)
    right_boundary = (right_mask > threshold).astype(np.uint8)

    contours, _ = cv2.findContours(
        left_boundary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )
    if len(contours) > 0:
        left_boundary = cv2.drawContours(
            left_boundary, contours, -1, (1), thickness=cv2.FILLED
        )

    contours, _ = cv2.findContours(
        right_boundary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )
    if len(contours) > 0:
        right_boundary = cv2.drawContours(
            right_boundary, contours, -1, (1), thickness=cv2.FILLED
        )

    return left_boundary, right_boundary


def lane_detection_pipeline(img, model_path="seg_model.pth"):
    # Load the model
    model = torch.load(model_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.GaussianBlur(img, (5, 5), 0)

    # Get predictions
    back, left, right = get_pred_for_mobilenet(model, img)[0]

    # Extract left and right boundaries
    left_boundary, right_boundary = extract_lane_boundaries(left, right)

    # Create ROI mask
    # roi_mask = (left_boundary | right_boundary).astype(np.uint8)

    # # Create a copy of the original image
    # image_with_roi = np.copy(img)

    # # Set all pixels outside the ROI to black
    # image_with_roi[roi_mask == 0] = [0, 0, 0]

    # # Overlay the left and right lane boundaries in red color
    # image_with_roi[left_boundary > 0] = [255, 0, 0]
    # image_with_roi[right_boundary > 0] = [0, 255, 0]

    # Find the coordinates of the left and right boundaries
    left_y, left_x = np.where(left_boundary > 0)
    right_y, right_x = np.where(right_boundary > 0)

    # Create a black background mask with the same dimensions as the image
    mask = np.zeros_like(img, dtype=np.uint8)

    # Create a polygon for the left boundary
    left_polygon = np.column_stack((left_x, left_y))

    # Create a polygon for the right boundary (reverse the order to close the gap)
    right_polygon = np.column_stack((right_x[::-1], right_y[::-1]))

    # Combine the left and right polygons to fill the region between them
    polygon_points = np.vstack((left_polygon, right_polygon))

    # Use fillPoly to fill the region between the left and right boundaries
    try:
        cv2.fillPoly(mask, [polygon_points], (255, 255, 255))
    except Exception as e:
        print(f"Error in cv2.fillPoly: {e}")
        return None

    # Bitwise-AND your original image and the mask
    result = cv2.bitwise_and(img, mask)

    # Save the resulting image
    # cv2.imwrite(save_path, result)
    return result, left, right

    # return result

In [5]:
cap = cv2.VideoCapture("../imgs/output.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    if cv2.waitKey(1) & 0xFF == ord("q") or ret == False:
        break
    result, left, right = lane_detection_pipeline(frame)
    # result_warped, result_hou = show_process_image(result)
    cv2.imshow("Frame", frame)
    cv2.imshow("Lane", result)
    # cv2.imshow("Warped", result_hou)
    # time.sleep(0.1)

cap.release()
cv2.destroyAllWindows()

In [ ]:
# %%timeit
# start_time = time.perf_counter()

folder = r"C:\Users\medha\AppData\Local\Temp\airsim_car"
images = os.listdir(folder)

for img_path in images:
    img = cv2.imread(os.path.join(folder, img_path), cv2.IMREAD_GRAYSCALE)
    steer, throttle = lane_detection_pipeline(img)
# end_time = time.perf_counter()
# print(f"Time taken: {end_time - start_time:.2f} seconds for {len(images)} images")

29.4 s ± 382 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
